In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
Setup complete ✅ (12 CPUs, 167.1 GB RAM, 38.1/235.7 GB disk)


In [3]:
#下載資料集
import gdown
import os
import shutil

#下載training_image.zip
gdown.download("https://drive.google.com/uc?export=download&id=1rf02EExkT0XtjfLZDA_t83y9wsmiZd0Z","/content/training_image.zip")
#下載training_label.zip
gdown.download("https://drive.google.com/uc?export=download&id=1EOyOA-m-qbukEHmQVkiXNm-Cqfnw8Ftc","/content/training_label.zip")
#下載訓練aortic_valve_colab.yaml
gdown.download("https://drive.google.com/u/0/uc?id=1DYmD-KQMwlUg_3KcFkkXTudL9wTzpKgF&export=download","/content/aortic_valve_colab.yaml")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1rf02EExkT0XtjfLZDA_t83y9wsmiZd0Z
From (redirected): https://drive.google.com/uc?export=download&id=1rf02EExkT0XtjfLZDA_t83y9wsmiZd0Z&confirm=t&uuid=75604a99-bf9e-4d93-b069-d863d0969dfa
To: /content/training_image.zip
100%|██████████| 1.83G/1.83G [00:07<00:00, 246MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1EOyOA-m-qbukEHmQVkiXNm-Cqfnw8Ftc
To: /content/training_label.zip
100%|██████████| 584k/584k [00:00<00:00, 152MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1DYmD-KQMwlUg_3KcFkkXTudL9wTzpKgF&export=download
To: /content/aortic_valve_colab.yaml
100%|██████████| 91.0/91.0 [00:00<00:00, 375kB/s]


'/content/aortic_valve_colab.yaml'

In [ ]:
#移動檔案
def find_patient_root(root):
    """往下找，直到找到含有 patientXXXX 的資料夾"""
    for dirpath, dirnames, filenames in os.walk(root):
        if any(d.startswith("patient") for d in dirnames):
            return dirpath
    return root  # fallback

# 解壓縮到固定資料夾
if not os.path.isdir("./training_image") and os.path.exists("training_image.zip"):
    os.makedirs("./training_image", exist_ok=True)
    !unzip -q training_image.zip -d ./training_image

if not os.path.isdir("./training_label") and os.path.exists("training_label.zip"):
    os.makedirs("./training_label", exist_ok=True)
    !unzip -q training_label.zip -d ./training_label

IMG_ROOT = find_patient_root("./training_image")
LBL_ROOT = find_patient_root("./training_label")

print("IMG_ROOT =", IMG_ROOT)
print("LBL_ROOT =", LBL_ROOT)
j
# 建立並清空輸出資料夾（若存在）
def ensure_clean_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path, exist_ok=True)

ensure_clean_dir("./datasets/train/images")
ensure_clean_dir("./datasets/train/labels")
ensure_clean_dir("./datasets/val/images")
ensure_clean_dir("./datasets/val/labels")

def move_patients(start, end, split):
    for i in range(start, end + 1):
        patient = f"patient{i:04d}"
        img_dir = os.path.join(IMG_ROOT, patient)
        lbl_dir = os.path.join(LBL_ROOT, patient)
        if not os.path.isdir(lbl_dir):
            continue

        for fname in os.listdir(lbl_dir):
            if not fname.endswith(".txt"):
                continue

            label_path = os.path.join(lbl_dir, fname)
            base, _ = os.path.splitext(fname)  # 取出檔名不含副檔名
            img_path = os.path.join(img_dir, base + ".png")
            if not os.path.exists(img_path):
                print(f"找不到對應圖片: {img_path}")
                continue

            shutil.copy2(img_path, f"./datasets/{split}/images/")
            shutil.copy2(label_path, f"./datasets/{split}/labels/")


# #法1:
# move_patients(1, 40, "train")
# move_patients(41, 50, "val")

#法2: 
# patient0001~0030 → train
move_patients(1, 10, "train")
move_patients(21, 50, "train")
# patient0031~0050 → val
move_patients(11, 20, "val")

print("完成移動！")

IMG_ROOT = ./training_image
LBL_ROOT = ./training_label
完成移動！


In [5]:
from ultralytics import YOLO
model = YOLO("yolo12n.pt")

model.train(
    data="./aortic_valve_colab.yaml",
    epochs=85,
    batch=32,
    imgsz=640,
    patience=40,
    device=0,
    close_mosaic=3,
    optimizer="AdamW",
    amp=True,
)

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=3, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./aortic_valve_colab.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=85, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=40, perspective=0.0, plots=True, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79e69cfde0c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolo12m.pt")

# model.train(
#     data="./aortic_valve_colab.yaml",
#     epochs=85,
#     batch=32,
#     imgsz=640,
#     patience=40,
#     device=0,
#     close_mosaic=3,
#     optimizer="AdamW",
#     amp=True,
# )

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolo11n.pt")

# model.train(
#     data="./aortic_valve_colab.yaml",
#     epochs=85,
#     batch=32,
#     imgsz=640,
#     patience=40,
#     device=0,
#     close_mosaic=3,
#     optimizer="AdamW",
#     amp=True,
# )

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolo11m.pt")

# model.train(
#     data="./aortic_valve_colab.yaml",
#     epochs=85,
#     batch=32,
#     imgsz=640,
#     patience=40,
#     device=0,
#     close_mosaic=3,
#     optimizer="AdamW",
#     amp=True,
# )

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolov10m.pt")

# model.train(
#     data="./aortic_valve_colab.yaml",
#     epochs=85,
#     batch=32,
#     imgsz=640,
#     patience=40,
#     device=0,
#     close_mosaic=3,
#     optimizer="AdamW",
#     amp=True,
# )

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolov9c.pt")

# model.train(
#     data="./aortic_valve_colab.yaml",
#     epochs=85,
#     batch=32,
#     imgsz=640,
#     patience=40,
#     device=0,
#     close_mosaic=3,
#     optimizer="AdamW",
#     amp=True,
# )

In [6]:
!zip -r '/content/train.zip' '/content/runs/detect/train' #打包訓練模型和結果
from google.colab import files
files.download('/content/train.zip')

  adding: content/runs/detect/train/ (stored 0%)
  adding: content/runs/detect/train/labels.jpg (deflated 35%)
  adding: content/runs/detect/train/BoxF1_curve.png (deflated 16%)
  adding: content/runs/detect/train/val_batch1_labels.jpg (deflated 12%)
  adding: content/runs/detect/train/train_batch5742.jpg (deflated 7%)
  adding: content/runs/detect/train/val_batch0_pred.jpg (deflated 12%)
  adding: content/runs/detect/train/val_batch2_labels.jpg (deflated 12%)
  adding: content/runs/detect/train/train_batch1.jpg (deflated 5%)
  adding: content/runs/detect/train/BoxR_curve.png (deflated 15%)
  adding: content/runs/detect/train/train_batch5741.jpg (deflated 11%)
  adding: content/runs/detect/train/train_batch2.jpg (deflated 3%)
  adding: content/runs/detect/train/val_batch0_labels.jpg (deflated 12%)
  adding: content/runs/detect/train/confusion_matrix.png (deflated 35%)
  adding: content/runs/detect/train/results.csv (deflated 61%)
  adding: content/runs/detect/train/results.png (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>